## **Questão 3**

Usando o trabalho dos Profs. Storopoli & Souza disponível no Google Colab: https://colab.research.google.com/github/storopoli/ciencia-de-dados/blob/main/notebooks/Aula_18_b_Redes_Neurais_com_PyTorch.ipynb#scrollTo=JjcTgN5kDvqS

Importando o torch:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

**Questões:**

3 - Modifique a classe ClassBin para que a rede tenha a seguinte arquitetura:

    a. Camada de Entrada: Mantém as 4 features de entrada.
    b. Primeira Camada Oculta: nn.Linear com 4 neurônios de entrada e 16 neurônios de saída, seguida por uma ativação ReLU.
    c. Segunda Camada Oculta: nn.Linear com 16 neurônios de entrada e 8 neurônios de saída, seguida por uma ativação ReLU.
    d. Camada de Saída: nn.Linear com 8 neurônios de entrada e 1 neurônio de saída, seguida por uma ativação Sigmoid.
    e. Remova todas as camadas de Dropout para uma nova arquitetura.

Todas serão respondidos no código abaixo: (Apenas letra f será repondida depois)

In [2]:
class ClassBinNova(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBinNova, self).__init__()
        self.linear1 = nn.Linear(4, 16)    # primeira hidden layer. Entrada 4(letra a), saída 16(letra b)
        self.linear2 = nn.Linear(16, 8)    # segunda hidden layer. Entrada 16, saída 8(letra c)
        self.linear3 = nn.Linear(8, 1)     # terceira hidden layer. Entrada 8, saída 1(letra d)
        self.sigmoid = nn.Sigmoid()        # função de ativação Sigmoid
        self.relu = nn.ReLU()              # função de ativação ReLU
        # Removendo as camadas de Dropout (letra e)
        
    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x

model = ClassBinNova()
print(model)

ClassBinNova(
  (linear1): Linear(in_features=4, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


    f. Treine o novo modelo com os mesmos hiperparâmetros (épocas, taxa de aprendizado, etc.) e compare a acurácia final (de treino e teste) com a do modelo original.

Criando o dataset do Titanic para testar:

In [3]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
titanic = sns.load_dataset('titanic')

feature_names = ['pclass', 'female', 'age', 'fare']
titanic['female'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic.dropna(subset=feature_names, inplace=True)  #891 para 714

X = titanic[feature_names].to_numpy()
y = titanic['survived'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123)

In [4]:
print('Tamanho de X_train: ', X_train.shape)
print('Tamanho de X_test: ', X_test.shape)
print('Tamanho de y_train: ', y_train.shape)
print('Tamanho de y_test: ', y_test.shape)

Tamanho de X_train:  (535, 4)
Tamanho de X_test:  (179, 4)
Tamanho de y_train:  (535,)
Tamanho de y_test:  (179,)


In [5]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 32  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.1

# Instânciar o Otimizador Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
from torch.utils.data import TensorDataset, DataLoader

# Converter X e y para torch.Tensor
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

# Um Dataset de Tensores - Array [X, y]
train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

Teste com o modelo antigo:

In [7]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.linear1 = nn.Linear(4, 4)    # primeira hidden layer
        self.dropout1 = nn.Dropout(0.2)   # dropout layer
        self.linear2 = nn.Linear(4, 4)    # segunda hidden layer
        self.dropout2 = nn.Dropout(0.2)   # dropout layer
        self.linear3 = nn.Linear(4, 1)    # terceira hidden layer
        self.dropout3 = nn.Dropout(0.2)   # dropout layer
        self.sigmoid = nn.Sigmoid()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        x = F.relu(self.linear3(x))
        x = self.dropout3(x)
        x = self.sigmoid(x)
        return x

model_old = ClassBin()
print(model_old)

ClassBin(
  (linear1): Linear(in_features=4, out_features=4, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear2): Linear(in_features=4, out_features=4, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (linear3): Linear(in_features=4, out_features=1, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)


In [8]:
for t in range(epochs):
    model_old.train() # Colocar o modelo em modo de treinamento (calcula os gradientes)
    
    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]
        
        # Propagação (Feed Forward)
        y_pred = model_old(X)
    
        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))
        
        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model_old.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.693
Época 2,
          Custo Treino: 0.693
Época 3,
          Custo Treino: 0.693
Época 4,
          Custo Treino: 0.693
Época 5,
          Custo Treino: 0.693
Época 6,
          Custo Treino: 0.693
Época 7,
          Custo Treino: 0.693
Época 8,
          Custo Treino: 0.693
Época 9,
          Custo Treino: 0.693
Época 10,
          Custo Treino: 0.693
Época 11,
          Custo Treino: 0.693
Época 12,
          Custo Treino: 0.693
Época 13,
          Custo Treino: 0.69
Época 14,
          Custo Treino: 0.693
Época 15,
          Custo Treino: 0.693
Época 16,
          Custo Treino: 0.693
Época 17,
          Custo Treino: 0.693
Época 18,
          Custo Treino: 0.693
Época 19,
          Custo Treino: 0.693
Época 20,
          Custo Treino: 0.693
Época 21,
          Custo Treino: 0.693
Época 22,
          Custo Treino: 0.693
Época 23,
          Custo Treino: 0.693
Época 24,
          Custo Treino: 0.693
Época 25,
          Custo Treino: 0.693
Época 26,


Treino com o modelo novo: Que foi criado usando ClassBinNova

In [9]:
for t in range(epochs):
    model.train() # Colocar o modelo em modo de treinamento (calcula os gradientes)
    
    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]
        
        # Propagação (Feed Forward)
        y_pred = model(X)
    
        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))
        
        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.619
Época 2,
          Custo Treino: 0.58
Época 3,
          Custo Treino: 0.63
Época 4,
          Custo Treino: 0.579
Época 5,
          Custo Treino: 0.738
Época 6,
          Custo Treino: 0.431
Época 7,
          Custo Treino: 0.624
Época 8,
          Custo Treino: 0.455
Época 9,
          Custo Treino: 0.584
Época 10,
          Custo Treino: 0.575
Época 11,
          Custo Treino: 0.375
Época 12,
          Custo Treino: 0.343
Época 13,
          Custo Treino: 0.528
Época 14,
          Custo Treino: 0.62
Época 15,
          Custo Treino: 0.437
Época 16,
          Custo Treino: 0.418
Época 17,
          Custo Treino: 0.661
Época 18,
          Custo Treino: 0.434
Época 19,
          Custo Treino: 0.355
Época 20,
          Custo Treino: 0.514
Época 21,
          Custo Treino: 0.519
Época 22,
          Custo Treino: 0.47
Época 23,
          Custo Treino: 0.468
Época 24,
          Custo Treino: 0.525
Época 25,
          Custo Treino: 0.59
Época 26,
    

Verificando a acurácia dos modelos:

Acurácia do modelo antigo:

In [10]:
model_old.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model_old(X_train)
train_pred = train_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model_old(X_test)
test_pred = test_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)

print(f"Acurácia de Treino: {train_acc}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Acurácia de Treino: 0.590654194355011

 ---------------------------

Acurácia de Teste: 0.6033519506454468


Acurácia do modelo novo:

In [11]:
model.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model(X_train)
train_pred = train_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model(X_test)
test_pred = test_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)

print(f"Acurácia de Treino: {train_acc}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Acurácia de Treino: 0.7925233840942383

 ---------------------------

Acurácia de Teste: 0.7821229100227356
